- https://huggingface.co/blog/stackllama
- https://github.com/huggingface/trl/tree/main/examples/research_projects/stack_llama_2/scripts


```
git clone git@github.com:huggingface/trl.git
cd trl
```

- run the scripts
    - `huggingface-cli login`
- TRL
    - 三个 research projects（https://github.com/huggingface/trl/tree/main/examples/research_projects）
        - stack_llama_2/scripts
        - tools
        - toxicity

## model & dataset

- stack exchange
    - 数据收集及标注
        - `score = log2 (1 + upvotes)` rounded to the nearest integer, plus 1 if the questioner accepted the answer (we assign a score of −1 if the number of upvotes is negative).

### accelerate

- accelerate config yaml
    - https://github.com/huggingface/trl/tree/main/examples/accelerate_configs

```
from accelerate import Accelerator
...

base_model = AutoModelForCausalLM.from_pretrained(
    ...
    device_map={"": Accelerator().local_process_index},
    ...
)
```

### zero3


```
accelerate launch --config-file examples/accelerate_configs/deepspeed_zero3.yaml examples/research_projects/stack_llama_2/scripts/sft_llama2.py \
    --output_dir="./sft" \
    --max_steps=500 \
    --logging_steps=10 \
    --save_steps=0.2 \
    --save_total_limit=2 \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=1 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing=False \
    --group_by_length=False \
    --learning_rate=1e-4 \
    --lr_scheduler_type="cosine" \
    --warmup_steps=100 \
    --weight_decay=0.05 \
    --optim="paged_adamw_32bit" \
    --bf16=True \
    --remove_unused_columns=False \
    --run_name="sft_llama2" \
    --report_to="wandb"
```

https://wandb.ai/loveresearch/huggingface/runs/jsn4syqr

- trl 中的 SFTTrainer 继承自 huggingface transformers 的 Trainer
- max_steps=500
    - 设置较小的 max_steps 可以用来做简单的 bug 测试；
    - steps 称之为 optimization steps，优化步；执行多少次优化；即反向传播，梯度计算与权重参数更新；
- learning_rate=1e-4 && lr_scheduler_type="cosine" && warmup_steps=100
    - warmup_steps 达到 100 steps 时达到 learning_rate=1e-4
    - warmup_steps 之前初始是 1e-5, 通过 100 步，线性到达 1e-4
    - 指定的 learning_rate 其实是 $\eta_{\text{max}}$，warmup 到达后，逐渐衰减；
- grad_norm

$$
\text{grad\_norm} = \sqrt{\sum_{i=1}^n \left(\frac{\partial L}{\partial w_i}\right)^2}
$$